# Presentation

1. Goal: Improve EDS (quantification, calibration, understanding)
2. Data: SEM Apreo, AZtec

3. Calibration (plot)
4. Quantification (table)


# Observations / questions

- Ga K alpha lines are one, not two

- How many use EDS SEM quantification? HyperSpy only does TEM
- How important is +- 5% error in calibration?


# Problems

- HyperSpy is large, complex, and not well documented
    - Legend om model plot
    - Hover info
    - Understanding how the background is fitted
- k-factors
    - Theoretically estimated
    - k-ratios?


# Further plan

- Write a report
- Organize the code, if it might be valuable for others
- Look more into factorless quantification
- More data

In [1]:
import hyperspy.api as hs
from utils.get_raw_data import locate_raw_data


In [2]:
%matplotlib qt

In [3]:
emsa = locate_raw_data()


In [16]:
s1 = hs.load(emsa[7], signal_type="EDS_TEM")
s1.add_elements(['As', 'Ga'])
s1.plot(xray_lines=True)

In [4]:
dispersion = 0.0100283 # from GaAs, Ga La and As Ka peaks
offset = 0.2113765074 # keV offset

# NB! k-factors are sorted alfabetically, so As first, then Ga.
# it is the same order as the x-ray lines in the family_lines list

# make function which takes a file name and returns the quantification results with three different calibrations
def quant_three_ways(filename, factors, s2_dispersion=0.0100283, s2_offset=0.2113765074, elements=['As', 'Ga'], family_lines=['As_Ka', 'Ga_Ka'], window_width=3, line_width=[5.0, 2.0], plot=False):
    bg_order = 12
    s1 = hs.load(filename, signal_type="EDS_TEM")
    s1.add_elements(elements)
    m1 = s1.create_model()
    m1.add_family_lines(family_lines)
    # m1.add_polynomial_background(order=bg_order)
    # m1.fit_background()
    m1.fit(bounded=True)
    bw1 = s1.estimate_background_windows(line_width=line_width)
    iw1 =  s1.estimate_integration_windows(windows_width=window_width)
    s1_intensities = s1.get_lines_intensity(background_windows=bw1, integration_windows=iw1)
    quant1 = s1.quantification(s1_intensities, 'CL', factors=factors)


    # print(s1.metadata)
    
    s2 = hs.load(filename, signal_type="EDS_TEM")
    s2.add_elements(elements)
    s2.axes_manager[-1].name = 'E'
    s2.axes_manager['E'].units = 'keV'
    s2.axes_manager['E'].scale = s2_dispersion
    s2.axes_manager['E'].offset = -s2_offset
    m2 = s2.create_model()
    m2.add_family_lines(family_lines)
    # m2.add_polynomial_background(order=bg_order)
    # m2.fit_background()
    m2.fit(bounded=True)
    bw2 = s2.estimate_background_windows(line_width=line_width)
    iw2 =  s2.estimate_integration_windows(windows_width=window_width)
    s2_intensities = s2.get_lines_intensity(background_windows=bw2, integration_windows=iw2)
    quant2 = s2.quantification(s2_intensities, 'CL', factors=factors)
    
    s3 = hs.load(filename, signal_type="EDS_TEM")
    s3.add_elements(elements)
    # m3 = s3.create_model(auto_background=False)
    m3 = s3.create_model()
    m3.add_family_lines(family_lines)
    # m3.add_polynomial_background(order=bg_order)
    # m3.fit_background()
    m3.fit()
    m3.calibrate_energy_axis(calibrate='scale')
    m3.calibrate_energy_axis(calibrate='offset')
    bw3 = s3.estimate_background_windows(line_width=line_width)
    iw3 =  s3.estimate_integration_windows(windows_width=window_width)
    s3_intensities = s3.get_lines_intensity(background_windows=bw3, integration_windows=iw3)
    quant3 = s3.quantification(s3_intensities, 'CL', factors=factors)

    if plot:
        m3.plot(True, legend='auto')
        # hs.plot.plot_spectra([s1,s2,s3], legend='auto')

    # return the quantification results, and the three signals
    return [quant1, quant2, quant3], [s1, s2, s3]

# pretty print of quantification results
def print_quant_table(title, quant_list, elements=['As', 'Ga']):
    print(f"\n{title}")
    print(f"\n{'':<11}| " + f"{'s1':<10}| " + f"{'s2':<10}| " + f"{'s3':<10}")
    for q in range(len(elements)):
        print(f"{elements[q]:<11}| " + f"{quant_list[0][q].data[0]:<10.2f}| " + f"{quant_list[1][q].data[0]:<10.2f}| " + f"{quant_list[2][q].data[0]:<10.2f}")

def print_dispersion_offset(s_list):
    print(f"\n{'':<11}| " + f"{'s1':<10}| " + f"{'s2':<10}| " + f"{'s3':<10}")
    print(f"{'dispersion':<11}| " + f"{s_list[0].axes_manager[-1].scale:<10.6f}| " + f"{s_list[1].axes_manager[-1].scale:<10.6f}| " + f"{s_list[2].axes_manager[-1].scale:<10.6f}")
    print(f"{'offset':<11}| " + f"{s_list[0].axes_manager[-1].offset:<10.6f}| " + f"{s_list[1].axes_manager[-1].offset:<10.6f}| " + f"{s_list[2].axes_manager[-1].offset:<10.6f}")



In [5]:
# s3 = hs.load(emsa[7], signal_type="EDS_TEM")
# s3.add_elements(elements)
# # m3 = s3.create_model(auto_background=False)
# m3 = s3.create_model()
# m3.add_family_lines(family_lines)
# # m3.add_polynomial_background(order=bg_order)
# # m3.fit_background()
# m3.fit()
# m3.calibrate_energy_axis(calibrate='scale')
# m3.calibrate_energy_axis(calibrate='offset')
# # bw3 = s3.estimate_background_windows(line_width=line_width)
# # iw3 =  s3.estimate_integration_windows(windows_width=window_width)
# # s3_intensities = s3.get_lines_intensity(background_windows=bw3, integration_windows=iw3)
# # quant3 = s3.quantification(s3_intensities, 'CL', factors=factors)

In [6]:
# for key in m3.active_components:
    # print(key.print_current_values())

In [7]:
# m3.plot?

In [8]:
factors5 = [1.210, 1.086] # 5kV k-factors, As then Ga on the L-series
family_lines=['Ga_La', 'As_La']
quant_list5, s_list5 = quant_three_ways(emsa[4], factors5, family_lines=family_lines, plot=False)
print_quant_table("Quantification results 5kV L-series", quant_list5)
# print_dispersion_offset(s_list5)

[########################################] | 100% Completed | 117.77 ms
[########################################] | 100% Completed | 107.12 ms
[########################################] | 100% Completed | 118.10 ms

Quantification results 5kV L-series

           | s1        | s2        | s3        
As         | 45.06     | 44.56     | 44.56     
Ga         | 54.94     | 55.44     | 55.44     


In [9]:
factors10 = [1.310, 1.223] # 10kV k-factors, Ga and As L-series
family_lines=['As_La', 'Ga_La']
quant_list10, s_list10 = quant_three_ways(emsa[5], factors10, family_lines=family_lines, plot=False)
print_quant_table("Quantification results 10kV L-series", quant_list10)
# print_dispersion_offset(s_list10)

[########################################] | 100% Completed | 111.93 ms
[########################################] | 100% Completed | 119.89 ms
[########################################] | 100% Completed | 119.06 ms

Quantification results 10kV L-series

           | s1        | s2        | s3        
As         | 100.00    | 100.00    | 100.00    
Ga         | 0.00      | 0.00      | 0.00      


In [10]:
factors15 = [1.331, 1.259] # 15kV k-factors, Ga and As L-series
family_lines=['Ga_La', 'As_La']
quant_list15, s_list15 = quant_three_ways(emsa[6], factors15, family_lines=family_lines)
print_quant_table("Quantification results 15kV L-series", quant_list15)
# print_dispersion_offset(s_list15)

[########################################] | 100% Completed | 110.87 ms
[########################################] | 100% Completed | 109.68 ms
[########################################] | 100% Completed | 119.56 ms

Quantification results 15kV L-series

           | s1        | s2        | s3        
As         | 5.00      | 5.33      | 4.70      
Ga         | 95.00     | 94.67     | 95.30     


In [17]:
# factors30 = [4.191, 1.986, 3.268, 1.505] # 30kV k-factors, Ga and As K-series
# family_lines=['As_Ka', 'C_Ka', 'Ga_Ka',  'O_Ka']
# elements=['As', 'C', 'Ga', 'O']
factors30 = [4.191, 3.268] # 30kV k-factors, Ga and As K-series
family_lines=['As_Ka', 'Ga_Ka']
elements=['As', 'Ga']
quant_list30, s_list30 = quant_three_ways(emsa[7], factors30, family_lines=family_lines, elements=elements, plot=True)
print_quant_table("Quantification results 30kV", quant_list30, elements)
print_dispersion_offset(s_list30)


[########################################] | 100% Completed | 106.52 ms
[########################################] | 100% Completed | 119.96 ms
[########################################] | 100% Completed | 114.45 ms

Quantification results 30kV

           | s1        | s2        | s3        
As         | 55.55     | 57.00     | 56.35     
Ga         | 44.45     | 43.00     | 43.65     

           | s1        | s2        | s3        
dispersion | 0.010000  | 0.010028  | 0.010003  
offset     | -0.200000 | -0.211377 | -0.203917 


In [12]:
# print huge table
def print_all_quants(Vaccs, quants, elements):
    names = ['s1', 's2', 's3']
    for i in range(len(Vaccs)):
        lines = ''
        for line in quants[i][0]:
            lines += f" {line.metadata.Sample.xray_lines[0]}"
        # lines = f"({quants[i][0][0].metadata.Sample.xray_lines[0]}, {quants[i][0][1].metadata.Sample.xray_lines[0]})"
        print(f"\n{Vaccs[i]} kV ({lines} ) \n{'-'*32}")
        print(f"{'':<8}| " + f"{elements[0]:<10}| " + f"{elements[1]:<10}")
        for j in range(len(names)):
            print(f"{names[j]:<8}| " + f"{quants[i][j][0].data[0]:<10.2f}| " + f"{quants[i][j][1].data[0]:<10.2f}")
        


In [13]:
Vaccs = [5, 10, 15, 30]
quants = [quant_list5, quant_list10, quant_list15, quant_list30]
elements = ['As', 'Ga']
dispersions = [s_list5, s_list10, s_list15, s_list30]
print_all_quants(Vaccs, quants, elements)


5 kV ( As_La Ga_La ) 
--------------------------------
        | As        | Ga        
s1      | 45.06     | 54.94     
s2      | 44.56     | 55.44     
s3      | 44.56     | 55.44     

10 kV ( As_La Ga_La ) 
--------------------------------
        | As        | Ga        
s1      | 100.00    | 0.00      
s2      | 100.00    | 0.00      
s3      | 100.00    | 0.00      

15 kV ( As_La Ga_La ) 
--------------------------------
        | As        | Ga        
s1      | 5.00      | 95.00     
s2      | 5.33      | 94.67     
s3      | 4.70      | 95.30     

30 kV ( As_Ka Ga_Ka ) 
--------------------------------
        | As        | Ga        
s1      | 55.55     | 44.45     
s2      | 57.00     | 43.00     
s3      | 56.35     | 43.65     
